In [5]:
%%writefile mult_matriz_mpi.c
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <mpi.h>

#define WIDTH 16
#define HEIGHT 16
#define MAX_VALUE 9

void init_matrix(int width, int height, int m[width][height])
{
    int i, j;
    for(i=0; i<width; i++){
        for(j=0; j<height; j++){
            m[i][j] = rand() % MAX_VALUE;
            }
    }
}

void print_matrix(int width, int height, int m[width][height])
{
    int i, j;

    for(i=0; i<width; i++)
    {
        printf("\n%d: ", i);
        for(j=0; j<height; j++)
            printf("%d ", m[i][j]);
    }
}

int main(int argc, char *argv[])
{
    srand(time(NULL));
    int i,j,k, rows_proc,numtasks;
    int width = WIDTH;
    int height = HEIGHT;
    int m1[width][height], m2[width][height], result[width][height];
    int myrank;
    MPI_Status status;



    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &myrank);
    
    //Num de processos em execução
    MPI_Comm_size(MPI_COMM_WORLD,&numtasks);
    //Numero de linhas a serem enviadas para cada processo
    rows_proc = HEIGHT / (numtasks-1);
    

    if (myrank == 0)
    {  // código para processo mestre

        int bloco1[rows_proc][width],bloco2[rows_proc][width];
        printf("\nMatrix 1:\n");
        init_matrix(width, height, m1);
        print_matrix(width, height, m1);

        printf("\nMatrix 2:\n");
        init_matrix(width, height, m2);
        print_matrix(width, height, m2);
        printf("\n\n");
        
        for(i=0;i<numtasks-1;i++) 
        {   
            //define o intervalo de linhas a serem enviadas ao processo i+1
            int inicio = i*rows_proc;
            int final = inicio+rows_proc;
            
            for(j = inicio ; j < final ; j++)
            {   
                for(k=0; k<width; k++)
                {   
                    bloco1[j][k] = m1[j][k];
                    bloco2[j][k] = m2[j][k];
                }
            }
            //envia as duas matrizes ao mesmo processo
            MPI_Send(bloco1, rows_proc*width, MPI_INT, i+1, 1, MPI_COMM_WORLD);
            MPI_Send(bloco2, rows_proc*width, MPI_INT, i+1, 2, MPI_COMM_WORLD);
            
        }

        //recebe a matriz resultante de cada processo
        for(i=0;i<numtasks-1;i++) {
            int inicio = i*rows_proc;
            int final = inicio+rows_proc;

            MPI_Recv(&bloco1,rows_proc*width,MPI_INT,i+1,3,MPI_COMM_WORLD, &status);
            
            //coloca cada matriz resultante na posição da matriz final
            for(j = inicio ; j < final ; j++)
                for(k = 0; k < width; k++){
                    result[j][k] = bloco1[j-inicio][k];
                    
                }
            
        }
        
        printf("\nResult:\n");
        print_matrix(width, height, result);
        
    }
    else
    {   // código para processos escravos
        int matriz_escravo[rows_proc][width];
        int rank;
        int bloco1[rows_proc][width],bloco2[rows_proc][width];
        
        MPI_Comm_rank(MPI_COMM_WORLD,&rank);

        //recebe as duas matrizes enviadas pelo processo mestre        
        MPI_Recv(&bloco1, rows_proc*width, MPI_INT, 0, 1, MPI_COMM_WORLD, &status); 
        MPI_Recv(&bloco2, rows_proc*width, MPI_INT, 0, 2, MPI_COMM_WORLD, &status); 
        
        /* Matrix multiplication 
        for (k=0; k<N; k++)
            for (i=0; i<rows; i++) {
            c[i][k] = 0.0;
                for (j=0; j<N; j++)
                c[i][k] = c[i][k] + a[i][j] * b[j][k];
        }
        for (i = 0; i < rows_proc; i++) //linhas de C
            for (j=0; j< width; j++) { //colunas de C
            c[j][i] = 0.0;
                for (k=0; k<rows_proc; k++) //colunas de A
                matriz_escravo[j][i] = matriz_escravo[j][i] + bloco1[j][k] * bloco2[k][i];
        }
        */
        
        for(i = 0; i < rows_proc; i++)
        {
            for(j=0; j< width; j++)
            {
                matriz_escravo[i][j] = bloco1[i][j] + bloco2[i][j];
            }
        }
        //envia as linhas resultantes da soma
        MPI_Send(matriz_escravo,rows_proc*width,MPI_INT,0,3,MPI_COMM_WORLD);
    
    }
    MPI_Finalize();
    return 0;
}

Overwriting mult_matriz_mpi.c


In [6]:
! mpicc -Wall mult_matriz_mpi.c -o mult